In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [10]:
cd dw_Matrix/

/content/drive/My Drive/Colab NoteBooks/dw_Matrix


In [11]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats,  model = DecisionTreeRegressor(max_depth=5)):
 X = df[ feats ].values
 y = df['prices_amountmin'].values
  
 scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
 return np.mean(scores), np.std(scores)

In [25]:
df['brand_cat'] = df['brand'].map(lambda x:str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [26]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [31]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [29]:
test = {'key' : 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [33]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)[0]['value']

['Men']

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [48]:
keys = set()


df['features_parsed'].map( lambda x: keys.update(x.keys()))
len(keys)

476

In [52]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  
 df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [53]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_wash', 'feat_design', 'feat_sport', 'feat_band length (in.)',
       'feat_packageweight', 'feat_audience',
       'feat_certifications and listings', 'feat_lens color',
       'feat_100% adidas authentic', 'feat_eye protection type'],
      dtype='object', length=526)

In [0]:

keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] *100

In [67]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

In [70]:
df['brand'] = df['brand'].map(lambda x:str(x).lower())
df [ df.brand == df.feat_brand].shape

(8846, 530)

In [105]:
feats_cat= [x for x in df.columns if 'cat' in x ]
feats_cat

['categories',
 'brand_cat',
 'feat_location - country',
 'feat_recommended location',
 'feat_shoe category',
 'feat_catalog',
 'feat_clothing category',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_multi pack indicator',
 'feat_certifications and listings',
 'feat_brand_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'feat_manufacturer part number_cat']

In [0]:
feats = [' ']

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat',]
#feats += [ 'feat_clothing category']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [103]:
 X = df[feats].values
 y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

print(result)
perm = PermutationImportance (m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)


(-57.16280496015928, 4.119744182709844)


Weight,Feature
0.2604 ± 0.0084,brand_cat
0.1115 ± 0.0137,feat_material_cat
0.0427 ± 0.0064,feat_gender_cat
0.0227 ± 0.0009,feat_brand_cat


In [85]:
df [df ['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'camping & hiking, fishing, golf, hunting, lacrosse, running, skateboarding, ski & snow, soccer, casual', 'style': 'shirts & tops', 'material': 'dri-fit', 'condition': 'new with tags', 'sleeve length': 'long sleeve'},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'men', 'shoe size': '8.5', 'size': '8.5', 'color': 'black', 'model': '653617 002', 'manufacturer part number': '653617 002', 'brand': 'nike', 'age group': 'adult'},
       {'style': 'basketball shoes', 'country/region of manufacture': 'vietnam', 'type': 'athletic-inspired', 'condition': 'new with box', 'pattern': 'solid', 'material': 'suede'},
       {'style': 'basketball shoes', 'condition': 'new with box'},
       {'style': 'basketball shoes', 'country/region of manufacture': 'vietnam', 'type': 'athletic-inspired', 'condition': 'new with box', 'pattern': 'solid', 'material': 'suede'}],
      dtype=object)

In [81]:
df['brand'].value_counts(normalize=True)

nike                   0.097210
puma                   0.033315
ralph lauren           0.028775
vans                   0.021116
new balance            0.020295
                         ...   
sendra                 0.000055
gitzo                  0.000055
coggs                  0.000055
stance                 0.000055
rush by gordon rush    0.000055
Name: brand, Length: 1732, dtype: float64

In [82]:
df [df ['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [106]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   day4.ipynb
	modified:   matrix_one/Day3.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	matrix_one/day4_meta.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git add -A

In [108]:
pwd

'/content/drive/My Drive/Colab NoteBooks/dw_Matrix'

In [109]:
ls

data/  day4.ipynb  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [110]:
ls

data/  day4.ipynb  HelloGithub.ipynb  LICENSE  matrix_one/  README.md
